# Current Plan
1. Create a function to calculate the distance between descriptions, use that function to naively classify new descriptions. probably looks something like a search function.
2. Grab a text classification model of hugging face and fine-tune it on the dataset.

In [ ]:
# setup requirements, which aren't loading in the venv for some reason
%pip install -r requirements.txt

In [20]:
# Set up data split
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

with open('data/Datas.csv') as file:
    data = pd.read_csv(file)
    
# Split data into training and testing sets in a stratified way, despite the minimum class only having 2 samples 
cat_count = data.value_counts('Category')
subcat_count = data.value_counts(['Category', 'Sub_Category'])

# handle the case where the minimum class has only 2 samples
data = data.groupby('Category').filter(lambda x: len(x) > 2) # this method works, but my pandas is pretty rusty. 

train, test = train_test_split(data, test_size=0.2, stratify=data['Category'], random_state=42) # random_state for reproducibility
train, val = train_test_split(train, test_size=0.3, stratify=train['Category'], random_state=77) 

## 1. Calculating distance 
Cosine similarity [shouldn't be affected by text length](https://nikoskalikis.medium.com/text-similarity-euclidian-distance-vs-cosine-similarity-3a1167f686a#:~:text=Cosine%20similarity%20is%20an%20important,value%20between%20%2D1%20and%201.), which is quite short here. 



In [ ]:
# Function to measure distance 

